In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import models, layers
import matplotlib.pyplot as plt
import pickle as pk

import splitfolders

In [ ]:
splitfolders.ratio(
    '/mnt/c/NN/Kidney/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone',
    output='/mnt/c/NN/Kidney/dataset',
    seed=7,
    ratio=(0.9, 0.05, 0.05)
)

In [9]:
train_datagen = ImageDataGenerator(rescale=1 / 255)
valid_datagen = ImageDataGenerator(rescale=1 / 255)
test_datagen = ImageDataGenerator(rescale=1 / 255)

folder_path = '/mnt/c/NN/Kidney/dataset/'

train_dataset = train_datagen.flow_from_directory(f'{folder_path}train',
                                                  target_size=(200, 200),
                                                  color_mode='grayscale', 
                                                  class_mode='categorical', 
                                                  batch_size=100,
                                                  )

test_dataset = test_datagen.flow_from_directory(f'{folder_path}test',
                                                target_size=(200, 200),
                                                class_mode='categorical',
                                                color_mode='grayscale',
                                                batch_size=100,
                                                shuffle=False
                                                )

valid_dataset = valid_datagen.flow_from_directory(f'{folder_path}val',
                                                  target_size=(200, 200),
                                                  class_mode='categorical',
                                                  batch_size=100,
                                                  color_mode='grayscale',
                                                  )


Found 11200 images belonging to 4 classes.
Found 626 images belonging to 4 classes.
Found 620 images belonging to 4 classes.


In [12]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=train_dataset.image_shape))
model.add(layers.MaxPool2D(2))

model.add(layers.Conv2D(32, (3 ,3), activation='relu'))
model.add(layers.MaxPool2D(2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(4, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 64)       0

In [13]:
METRICS = [
    'accuracy',
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
]

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=METRICS)

In [14]:
data = model.fit_generator(train_dataset , validation_data=valid_dataset, epochs=3)

/tmp/ipykernel_3585/1470813877.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  data = model.fit_generator(train_dataset,validation_data=valid_dataset,epochs=3)


Epoch 1/3


Copying files: 3298 files [50:02,  1.10 files/s]
Copying files: 277 files [47:03, 10.19s/ files]


112/112 [==============================] - 326s 3s/step - loss: 1.0971 - accuracy: 0.5696 - precision: 0.6698 - recall: 0.3889 - val_loss: 0.8793 - val_accuracy: 0.6145 - val_precision: 0.7526 - val_recall: 0.4613
Epoch 2/3
112/112 [==============================] - 321s 3s/step - loss: 0.5793 - accuracy: 0.7737 - precision: 0.8170 - recall: 0.7221 - val_loss: 0.3292 - val_accuracy: 0.8613 - val_precision: 0.8804 - val_recall: 0.8548
Epoch 3/3
112/112 [==============================] - 304s 3s/step - loss: 0.2012 - accuracy: 0.9362 - precision: 0.9411 - recall: 0.9324 - val_loss: 0.0359 - val_accuracy: 0.9952 - val_precision: 0.9952 - val_recall: 0.9952


In [17]:
test_acc = model.evaluate(test_dataset, verbose=2)

7/7 - 8s - loss: 0.0429 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9952 - 8s/epoch - 1s/step


In [ ]:
filename = '../project/prediction_models/k.bin'
pk.dump(model, open(filename, 'wb'))
